In [21]:
import pandas as pd
import numpy as np
import joblib
import os
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor

In [7]:
df = pd.read_csv("ENB2012_data.csv")

In [8]:
df.head()

,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0,15.55,21.33
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0,15.55,21.33
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0,15.55,21.33
3,0.98,514.5,294.0,110.25,7.0,5,0.0,0,15.55,21.33
4,0.90,563.5,318.5,122.50,7.0,2,0.0,0,20.84,28.28


In [9]:
train, test = train_test_split(df, test_size=0.2)
X_train = np.array(train.iloc[:,:-2])
y_train = np.array(train.iloc[:,-2:])
X_test = np.array(test.iloc[:,:-2])
y_test = np.array(test.iloc[:,-2:])

In [10]:
max_depth = 30
regr_multirf = MultiOutputRegressor(RandomForestRegressor(n_estimators=100,
                                                          max_depth=max_depth,
                                                          random_state=0))
regr_multirf.fit(X_train, y_train)

y_multirf = regr_multirf.predict(X_test)

In [11]:
# save the model to disk
model_name = 'multi_output_model.sav'
joblib.dump(regr_multirf, model_name)

['multi_output_model.sav']

In [12]:
X_test

array([[7.100e-01, 7.105e+02, 2.695e+02, ..., 4.000e+00, 2.500e-01,
        2.000e+00],
       [6.900e-01, 7.350e+02, 2.940e+02, ..., 3.000e+00, 1.000e-01,
        5.000e+00],
       [7.600e-01, 6.615e+02, 4.165e+02, ..., 3.000e+00, 2.500e-01,
        1.000e+00],
       ...,
       [9.800e-01, 5.145e+02, 2.940e+02, ..., 5.000e+00, 4.000e-01,
        1.000e+00],
       [6.900e-01, 7.350e+02, 2.940e+02, ..., 4.000e+00, 4.000e-01,
        3.000e+00],
       [6.400e-01, 7.840e+02, 3.430e+02, ..., 3.000e+00, 1.000e-01,
        2.000e+00]])

In [13]:
loaded_model = joblib.load("multi_output_model.sav")

In [14]:
prediction = loaded_model.predict(X_test)

In [34]:
import json
def load_tester(path):
    with open(path) as f:
        data = json.load(f)
    print(data)
    return np.asarray(list(data.values())).reshape(1,-1)   #reshape gives the input shape we need

In [35]:
new_data = load_tester("new_building.json")


{'Relative Compactness': 0.74, 'Surface Area': 588.0, 'Wall Area': 294.0, 'Roof Area': 147.0, 'Overall Height': 7.0, 'Orientation': 5, 'Glazing Area': 0.1, 'Glazing Area Distribution': 1}


In [36]:
new_data

array([[7.40e-01, 5.88e+02, 2.94e+02, 1.47e+02, 7.00e+00, 5.00e+00,
        1.00e-01, 1.00e+00]])

In [44]:
loads = ['Heating','Cooling']
#output is a 2d array that converts into a list of lists, need to take the first value
pred_values = loaded_model.predict(new_data).tolist()[0]  

In [46]:
response_to_api_call = dict(zip(loads, pred_values))

In [47]:
print(response_to_api_call)

{'Heating': 21.106699999999993, 'Cooling': 24.3448}


In [50]:
import pandas as pd
import numpy as np
import joblib
import os
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
import json

def load_tester(path):
    with open(path) as f:
        data = json.load(f)
    return np.asarray(list(data.values())).reshape(1,-1)  


def predict_loads(json_file):
    new_data = load_tester(json_file)
    loaded_model = joblib.load("multi_output_model.sav")
    loads = ['Heating','Cooling']
    pred_values = loaded_model.predict(new_data).tolist()[0] 
    response_to_api_call = dict(zip(loads, pred_values))
    print(response_to_api_call)

In [51]:
predict_loads("new_building.json")

{'Heating': 21.106699999999993, 'Cooling': 24.3448}


In [56]:
%%file regression_model.py
import pandas as pd
import numpy as np
import joblib
import json
import sys

def load_tester(file):
    with open(file) as f:
        data = json.load(f)
    return np.asarray(list(data.values())).reshape(1,-1)  


def predict_loads(json_file):
    new_data = load_tester(json_file)
    loaded_model = joblib.load("multi_output_model.sav")
    loads = ['Heating','Cooling']
    pred_values = loaded_model.predict(new_data).tolist()[0] 
    response_to_api_call = dict(zip(loads, pred_values))
    print(response_to_api_call)

new_data = sys.argv[1]
predict_loads(new_data)

Overwriting regression_model.py


In [57]:
import requests

In [2]:
import flask

In [80]:
%%file app_flask.py
from flask import Flask, request
from werkzeug.utils import secure_filename

app = Flask(__name__)

from flask import Flask, jsonify, request

app = Flask(__name__)

incomes = [
  { 'description': 'salary', 'amount': 5000 }
]

@app.route('/incomes')
def get_incomes():
  return jsonify(incomes)


@app.route('/incomes', methods=['POST'])
def add_income():
  incomes.append(request.get_json())
  return '', 204

if __name__=="__main__":
    app.run(port=5000, debug=True)

Overwriting app_flask.py


In [75]:
os.listdir()

['ENB2012_data.csv',
 'new_building.json',
 'Dockerfile',
 'multi_output_model.sav',
 'MultiOutput_RandomForest_Regressor.ipynb',
 'app_flask.py',
 'regression_model.py',
 '.ipynb_checkpoints']

In [42]:
%%file multioutput_rf_flask_app.py
from flask import Flask, request
from werkzeug.utils import secure_filename
import pandas as pd
import numpy as np
import joblib
import json
import sys

app = Flask(__name__)

from flask import Flask, jsonify, request

app = Flask(__name__)

@app.route("/")
def hello_world():
    return "Hello, this is the app for Energy Load Estimates.\n"

building_parameters = []

# @app.route('/building_parameters', methods=['POST'])
# def receive_building_parameters():
#     building_parameters.append(request.get_json())
#     #return '', 204
#     return jsonify(building_parameters[-1])

@app.route('/get_estimates', methods=['POST'])
def predict_loads():
    building_parameters.append(request.get_json())
    new_data = np.asarray(list(building_parameters[-1].values())).reshape(1,-1)
    loaded_model = joblib.load("multi_output_model.sav")
    loads = ['Heating','Cooling']
    pred_values = loaded_model.predict(new_data).tolist()[0] 
    response_to_api_call = dict(zip(loads, pred_values))
    return jsonify(response_to_api_call)

if __name__=="__main__":
    app.run(port=5000, host='0.0.0.0',debug=True)

Overwriting multioutput_rf_flask_app.py


$ curl http://127.0.0.1:5000/

Hello, this is the app for Energy Load Estimates

$ curl -X POST -H "Content-Type: application/json" -d '{
    "Relative Compactness": 0.74,
    "Surface Area": 588.00,
    "Wall Area": 294.00,
    "Roof Area": 147.00,
    "Overall Height": 7.00,
    "Orientation": 5,
    "Glazing Area": 0.10,
    "Glazing Area Distribution": 1
}' http://0.0.0.0:5000/get_estimates

In [23]:
est = [
  {
    "Glazing Area": 0.1,
    "Glazing Area Distribution": 1,
    "Orientation": 5,
    "Overall Height": 7.0,
    "Relative Compactness": 0.74,
    "Roof Area": 147.0,
    "Surface Area": 588.0,
    "Wall Area": 294.0
  },
  {
    "Glazing Area": 0.2,
    "Glazing Area Distribution": 1,
    "Orientation": 4,
    "Overall Height": 7.0,
    "Relative Compactness": 0.74,
    "Roof Area": 147.0,
    "Surface Area": 588.0,
    "Wall Area": 294.0
  }
]

In [ ]:
$ curl http://0.0.0.0:5000/

$ curl -X POST -H "Content-Type: application/json" -d '{
    "Relative Compactness": 0.74,
    "Surface Area": 588.00,
    "Wall Area": 294.00,
    "Roof Area": 147.00,
    "Overall Height": 7.00,
    "Orientation": 5,
    "Glazing Area": 0.10,
    "Glazing Area Distribution": 1
}' http://0.0.0.0:5000/get_estimates